# End-to-End 시나리오 — 일별 배치부터 장애 복구까지

이 노트북에서는 지금까지 배운 모든 내용을 **실전 시나리오**로 종합 실습합니다.

### 시나리오

온라인 쇼핑몰의 주문 데이터 파이프라인:

1. **일별 배치 적재** — 매일 새로운 주문 데이터 INSERT
2. **상태 업데이트** — 배송 완료된 주문 UPDATE
3. **스키마 변경** — 새 컬럼 추가 (Schema Evolution)
4. **컴팩션** — Small File Problem 해결
5. **스냅샷 정리** — 유지보수 작업
6. **장애 복구** — 실수로 데이터 삭제 후 Time Travel로 복구

## 환경 설정

In [47]:
import sys

sys.path.append("..")

import time
from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import (
    show_tree,
    snapshot_tree,
    diff_tree,
    count_files,
    total_size,
)

In [48]:
spark = create_spark_session()

TABLE_NAME = "demo.lab.e2e_orders"
TABLE_PATH = "/home/jovyan/data/warehouse/lab/e2e_orders"

Spark + Iceberg 세션 준비 완료 (warehouse: file:///home/jovyan/data/warehouse)


---
## Phase 1: 테이블 생성 + 일별 배치 적재

월별 Hidden Partitioning으로 테이블을 생성하고, 5일치 배치 데이터를 순차적으로 적재합니다.

In [49]:
spark.sql(f"DROP TABLE IF EXISTS {TABLE_NAME}")

spark.sql(f"""
CREATE TABLE {TABLE_NAME} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
""")

print(f"테이블 생성 완료: {TABLE_NAME}")
print("파티셔닝: months(order_date)")

테이블 생성 완료: demo.lab.e2e_orders
파티셔닝: months(order_date)


In [50]:
# Day 1~5: 일별 배치 적재 시뮬레이션
daily_batches = [
    {"day": 1, "start": "2024-01-01", "end": "2024-01-31", "records": 100},
    {"day": 2, "start": "2024-01-01", "end": "2024-01-31", "records": 80},
    {"day": 3, "start": "2024-02-01", "end": "2024-02-28", "records": 120},
    {"day": 4, "start": "2024-02-01", "end": "2024-02-28", "records": 90},
    {"day": 5, "start": "2024-03-01", "end": "2024-03-31", "records": 110},
]

offset = 1
for batch in daily_batches:
    orders = generate_orders(
        num_records=batch["records"],
        seed=batch["day"],
        start_date=batch["start"],
        end_date=batch["end"],
        id_offset=offset,
    )
    df = to_spark_df(spark, orders)
    df.writeTo(TABLE_NAME).append()
    offset += batch["records"]
    print(
        f"Day {batch['day']}: {batch['records']}건 적재 ({batch['start']} ~ {batch['end']})"
    )

total = spark.sql(f"SELECT COUNT(*) FROM {TABLE_NAME}").collect()[0][0]
print(f"\n총 레코드: {total}건")
print(f"파일 수: {count_files(TABLE_PATH)}")

Day 1: 100건 적재 (2024-01-01 ~ 2024-01-31)
Day 2: 80건 적재 (2024-01-01 ~ 2024-01-31)
Day 3: 120건 적재 (2024-02-01 ~ 2024-02-28)
Day 4: 90건 적재 (2024-02-01 ~ 2024-02-28)
Day 5: 110건 적재 (2024-03-01 ~ 2024-03-31)

총 레코드: 500건
파일 수: 5


In [51]:
# 현재 상태 확인
print("Phase 1 완료 — 파일 구조:")
print("=" * 60)
show_tree(TABLE_PATH, max_depth=3)

print("\n스냅샷 히스토리:")
spark.sql(f"""
SELECT snapshot_id, committed_at, operation,
       summary['added-data-files'] as added_files,
       summary['added-records'] as added_records
FROM {TABLE_NAME}.snapshots
ORDER BY committed_at
""").show(truncate=False)

Phase 1 완료 — 파일 구조:
├── data/
│   ├── order_date_month=2024-01/
│   │   ├── 00000-69-6b680a7c-560e-4038-8c12-bdb7ae56aed9-00001.parquet  (3.0 KB)
│   │   └── 00000-75-4557e7d1-e2e2-4063-b7a9-bdc43bc7a0d7-00001.parquet  (2.8 KB)
│   ├── order_date_month=2024-02/
│   │   ├── 00000-81-28b87232-ee0b-49b7-9eb7-32eddef66fe2-00001.parquet  (3.1 KB)
│   │   └── 00000-87-d0f8bf21-cded-49f6-8166-8d245926ea36-00001.parquet  (2.9 KB)
│   └── order_date_month=2024-03/
│       └── 00000-93-ece02ff2-1055-4c78-98ca-edba739262b1-00001.parquet  (3.0 KB)
└── metadata/
    ├── 2c685101-e974-4ec3-afef-11d108ac0dca-m0.avro  (7.1 KB)
    ├── 9178965a-fd48-4b92-8038-16e764ea9871-m0.avro  (7.1 KB)
    ├── 9f5ba179-81f1-4333-bec4-bdb29c164c8d-m0.avro  (7.1 KB)
    ├── ab18da90-7ea9-46ea-b4e1-c98cd1b94214-m0.avro  (7.1 KB)
    ├── c8f0742e-15a5-450f-b7cf-c742db199572-m0.avro  (7.1 KB)
    ├── snap-3246290473534841970-1-9178965a-fd48-4b92-8038-16e764ea9871.avro  (4.4 KB)
    ├── snap-6067149720729449558-1-9f5ba17

---
## Phase 1.5: 중복 데이터 유입과 정리

운영에서는 동일 주문이 재전송되어 중복 키가 들어올 수 있습니다.
업데이트 전에 중복을 탐지/정리하여 이후 단계의 정합성을 보장합니다.


In [52]:
# 3건을 의도적으로 중복 유입
dup_rows = spark.sql(f"""
SELECT order_id, customer_id, product_name, order_date, amount, status
FROM {TABLE_NAME}
ORDER BY order_id
LIMIT 3
""")
dup_rows.writeTo(TABLE_NAME).append()

dup_keys = spark.sql(f"""
SELECT COUNT(*)
FROM (
    SELECT order_id
    FROM {TABLE_NAME}
    GROUP BY order_id
    HAVING COUNT(*) > 1
) t
""").collect()[0][0]
print(f"중복 주입 후 중복 order_id 개수: {dup_keys}")

# order_id 기준으로 1건만 남기도록 정리
# cache()에 의존하지 않고, 임시 Iceberg 테이블에 먼저 물질화해 lineage를 완전히 분리
TMP_DEDUP_TABLE = f"{TABLE_NAME}_dedup_tmp"
spark.sql(f"DROP TABLE IF EXISTS {TMP_DEDUP_TABLE}")
total_before_dedup = spark.sql(
    f"SELECT COUNT(*) FROM {TABLE_NAME}"
).collect()[0][0]

spark.sql(f"""
CREATE TABLE {TMP_DEDUP_TABLE}
USING ICEBERG
AS
SELECT order_id, customer_id, product_name, order_date, amount, status
FROM (
    SELECT *,
           ROW_NUMBER() OVER (
               PARTITION BY order_id
               ORDER BY order_date DESC, status DESC
           ) AS rn
    FROM {TABLE_NAME}
) ranked
WHERE rn = 1
""")

dedup_result_count = spark.sql(
    f"SELECT COUNT(*) FROM {TMP_DEDUP_TABLE}"
).collect()[0][0]
dedup_removed_rows = total_before_dedup - dedup_result_count
print(f"중복 정리 전 총 레코드 수: {total_before_dedup}")
print(f"중복 제거 대상 레코드 수: {dedup_removed_rows}")
print(f"중복 정리 후 예상 레코드 수: {dedup_result_count}")

spark.sql(f"DELETE FROM {TABLE_NAME} WHERE true")
spark.sql(f"INSERT INTO {TABLE_NAME} SELECT * FROM {TMP_DEDUP_TABLE}")
spark.sql(f"DROP TABLE IF EXISTS {TMP_DEDUP_TABLE}")

dup_keys_after = spark.sql(f"""
SELECT COUNT(*)
FROM (
    SELECT order_id
    FROM {TABLE_NAME}
    GROUP BY order_id
    HAVING COUNT(*) > 1
) t
""").collect()[0][0]
print(f"중복 정리 후 중복 order_id 개수: {dup_keys_after}")


중복 주입 후 중복 order_id 개수: 3
중복 정리 전 총 레코드 수: 503
중복 제거 대상 레코드 수: 3
중복 정리 후 예상 레코드 수: 500
중복 정리 후 중복 order_id 개수: 0



### 관찰 포인트 — 중복 데이터 정리

- 중복 유입을 조기에 정리하면 UPDATE/DELETE/집계 결과 왜곡을 예방할 수 있습니다.
- 운영 파이프라인에는 배치 적재 직후 중복 탐지 쿼리를 자동화하는 것이 안전합니다.


---
## Phase 2: 상태 업데이트 (COW)

배송이 완료된 주문의 상태를 `shipped`로, 일부 주문을 `cancelled`로 업데이트합니다.

In [53]:
total_before_phase2 = spark.sql(
    f"SELECT COUNT(*) FROM {TABLE_NAME}"
).collect()[0][0]
if total_before_phase2 == 0:
    raise RuntimeError(
        "테이블이 비어 있습니다. Phase 1의 일별 배치 적재 셀(Cell 6)을 먼저 실행하세요."
    )

before = snapshot_tree(TABLE_PATH)

# pending → shipped
pending_target = spark.sql(f"""
SELECT COUNT(*)
FROM {TABLE_NAME}
WHERE status = 'pending' AND order_date < '2024-02-01'
""").collect()[0][0]
print(f"shipped 업데이트 대상: {pending_target}건")

spark.sql(f"""
UPDATE {TABLE_NAME}
SET status = 'shipped'
WHERE status = 'pending' AND order_date < '2024-02-01'
""")
shipped = spark.sql(
    f"SELECT COUNT(*) FROM {TABLE_NAME} WHERE status = 'shipped'"
).collect()[0][0]
print(f"shipped 상태 업데이트 완료 (현재 shipped: {shipped}건)")

# 일부 cancelled
cancel_target = spark.sql(f"""
SELECT COUNT(*)
FROM {TABLE_NAME}
WHERE amount < 150 AND order_date >= '2024-03-01'
""").collect()[0][0]
print(f"cancelled 업데이트 대상: {cancel_target}건")

spark.sql(f"""
UPDATE {TABLE_NAME}
SET status = 'cancelled'
WHERE amount < 150 AND order_date >= '2024-03-01'
""")
cancelled = spark.sql(
    f"SELECT COUNT(*) FROM {TABLE_NAME} WHERE status = 'cancelled'"
).collect()[0][0]
print(f"cancelled 상태 업데이트 완료 (현재 cancelled: {cancelled}건)")

after = snapshot_tree(TABLE_PATH)
print("\n파일 변경 사항 (COW 방식):")
diff_tree(before, after)


shipped 업데이트 대상: 32건
shipped 상태 업데이트 완료 (현재 shipped: 129건)
cancelled 업데이트 대상: 11건
cancelled 상태 업데이트 완료 (현재 cancelled: 110건)

파일 변경 사항 (COW 방식):

[+] 추가된 파일 (10개):
    + data/order_date_month=2024-01/00000-120-5afec7b2-bc83-41bf-a590-06dd94582164-00001.parquet  (3.6 KB)
    + data/order_date_month=2024-03/00000-125-b3c638fc-a05f-42b0-941d-dc8f6f9b03e6-00001.parquet  (3.0 KB)
    + metadata/7bc98a4b-d7b9-4646-9b0b-5970c1deca2e-m0.avro  (7.3 KB)
    + metadata/7bc98a4b-d7b9-4646-9b0b-5970c1deca2e-m1.avro  (7.1 KB)
    + metadata/af325f71-a8d3-453f-bcf0-5ce994623c9c-m0.avro  (7.2 KB)
    + metadata/af325f71-a8d3-453f-bcf0-5ce994623c9c-m1.avro  (7.1 KB)
    + metadata/snap-6696529076417919547-1-7bc98a4b-d7b9-4646-9b0b-5970c1deca2e.avro  (4.2 KB)
    + metadata/snap-95883035658351778-1-af325f71-a8d3-453f-bcf0-5ce994623c9c.avro  (4.3 KB)
    + metadata/v10.metadata.json  (10.5 KB)
    + metadata/v11.metadata.json  (11.5 KB)

[~] 크기 변경된 파일 (1개):
    ~ metadata/version-hint.text  (1 B -> 2 B)



In [54]:
# 상태별 분포 확인
total_for_distribution = spark.sql(
    f"SELECT COUNT(*) FROM {TABLE_NAME}"
).collect()[0][0]

print("주문 상태 분포:")
if total_for_distribution == 0:
    print("현재 테이블에 데이터가 없습니다. Phase 1(셀 6)과 Phase 2(셀 9)를 순서대로 실행하세요.")
else:
    spark.sql(f"""
    SELECT status, COUNT(*) as count
    FROM {TABLE_NAME}
    GROUP BY status
    ORDER BY count DESC
    """).show()

주문 상태 분포:
+----------+-----+
|    status|count|
+----------+-----+
|   shipped|  127|
|processing|  122|
| cancelled|  110|
| completed|   90|
|   pending|   51|
+----------+-----+



---
## Phase 3: 스키마 변경 (Schema Evolution)

비즈니스 요구사항 변경: `region` 컬럼을 추가합니다.

In [55]:
# region 컬럼 추가
spark.sql(f"ALTER TABLE {TABLE_NAME} ADD COLUMN region STRING")
print("region 컬럼 추가 완료")

# 스키마 확인
spark.sql(f"DESCRIBE {TABLE_NAME}").show()

region 컬럼 추가 완료
+--------------+------------------+-------+
|      col_name|         data_type|comment|
+--------------+------------------+-------+
|      order_id|            bigint|   null|
|   customer_id|            bigint|   null|
|  product_name|            string|   null|
|    order_date|              date|   null|
|        amount|     decimal(10,2)|   null|
|        status|            string|   null|
|        region|            string|   null|
|              |                  |       |
|# Partitioning|                  |       |
|        Part 0|months(order_date)|       |
+--------------+------------------+-------+



In [56]:
# 기존 데이터에 region 값 업데이트
spark.sql(f"""
UPDATE {TABLE_NAME}
SET region = CASE
    WHEN customer_id % 3 = 0 THEN 'Seoul'
    WHEN customer_id % 3 = 1 THEN 'Busan'
    ELSE 'Jeju'
END
""")

print("기존 데이터 region 업데이트 완료")
spark.sql(f"""
SELECT region, COUNT(*) as count
FROM {TABLE_NAME}
GROUP BY region
""").show()

기존 데이터 region 업데이트 완료
+------+-----+
|region|count|
+------+-----+
| Seoul|  146|
|  Jeju|  193|
| Busan|  161|
+------+-----+



In [57]:
# 새 데이터는 region 포함하여 삽입
from pyspark.sql.functions import lit, when, col

orders_new = generate_orders(
    num_records=100,
    seed=99,
    start_date="2024-03-15",
    end_date="2024-03-31",
    id_offset=501,
)
df_new = to_spark_df(spark, orders_new)
df_new = df_new.withColumn(
    "region",
    when(col("customer_id") % 3 == 0, lit("Seoul"))
    .when(col("customer_id") % 3 == 1, lit("Busan"))
    .otherwise(lit("Jeju")),
)
df_new.writeTo(TABLE_NAME).append()

total = spark.sql(f"SELECT COUNT(*) FROM {TABLE_NAME}").collect()[0][0]
print(f"새 데이터 100건 추가 (region 포함). 총 레코드: {total}건")

새 데이터 100건 추가 (region 포함). 총 레코드: 600건


---
## Phase 4: 컴팩션

여러 번의 INSERT/UPDATE로 파일이 많아졌습니다. Sort 전략으로 컴팩션합니다.

In [58]:
files_before = count_files(TABLE_PATH)
size_before = total_size(TABLE_PATH)
print(f"컴팩션 전: 파일 {files_before}개, 크기 {size_before:,} bytes")

# Sort 컴팩션 (order_date 기준)
spark.sql(f"""
CALL demo.system.rewrite_data_files(
    table => '{TABLE_NAME}',
    strategy => 'sort',
    sort_order => 'order_date ASC NULLS LAST'
)
""").show(truncate=False)

files_after = count_files(TABLE_PATH)
size_after = total_size(TABLE_PATH)
print(f"컴팩션 후: 파일 {files_after}개, 크기 {size_after:,} bytes")
print(f"파일 수 변화: {files_before} → {files_after}")

컴팩션 전: 파일 15개, 크기 377,788 bytes
+--------------------------+----------------------+---------------------+-----------------------+
|rewritten_data_files_count|added_data_files_count|rewritten_bytes_count|failed_data_files_count|
+--------------------------+----------------------+---------------------+-----------------------+
|0                         |0                     |0                    |0                      |
+--------------------------+----------------------+---------------------+-----------------------+

컴팩션 후: 파일 15개, 크기 377,788 bytes
파일 수 변화: 15 → 15


---
## Phase 5: 스냅샷 정리 (유지보수)

지금까지 생성된 스냅샷 중 최근 2개만 남기고 정리합니다.

In [59]:
# 현재 스냅샷 수 확인
snapshots = spark.sql(f"SELECT * FROM {TABLE_NAME}.snapshots").collect()
print(f"현재 스냅샷 수: {len(snapshots)}")

# 최근 2개만 유지
spark.sql(f"""
CALL demo.system.expire_snapshots(
    table => '{TABLE_NAME}',
    retain_last => 2
)
""").show(truncate=False)

snapshots_after = spark.sql(f"SELECT * FROM {TABLE_NAME}.snapshots").collect()
print(f"정리 후 스냅샷 수: {len(snapshots_after)}")

현재 스냅샷 수: 12
+------------------------+-----------------------------------+-----------------------------------+----------------------------+----------------------------+------------------------------+
|deleted_data_files_count|deleted_position_delete_files_count|deleted_equality_delete_files_count|deleted_manifest_files_count|deleted_manifest_lists_count|deleted_statistics_files_count|
+------------------------+-----------------------------------+-----------------------------------+----------------------------+----------------------------+------------------------------+
|0                       |0                                  |0                                  |0                           |0                           |0                             |
+------------------------+-----------------------------------+-----------------------------------+----------------------------+----------------------------+------------------------------+

정리 후 스냅샷 수: 12


In [60]:
# Manifest 재작성
spark.sql(f"""
CALL demo.system.rewrite_manifests(
    table => '{TABLE_NAME}'
)
""").show(truncate=False)

print("유지보수 완료: Expire Snapshots + Rewrite Manifests")

+-------------------------+---------------------+
|rewritten_manifests_count|added_manifests_count|
+-------------------------+---------------------+
|2                        |1                    |
+-------------------------+---------------------+

유지보수 완료: Expire Snapshots + Rewrite Manifests


In [61]:
# 유지보수 후 최종 상태
print("Phase 5 완료 — 유지보수 후 테이블 상태:")
print("=" * 60)
print(f"레코드 수: {spark.sql(f'SELECT COUNT(*) FROM {TABLE_NAME}').collect()[0][0]}")
print(f"파일 수: {count_files(TABLE_PATH)}")
print(f"스냅샷 수: {len(spark.sql(f'SELECT * FROM {TABLE_NAME}.snapshots').collect())}")

print("\n파일 구조:")
show_tree(TABLE_PATH, max_depth=3)

Phase 5 완료 — 유지보수 후 테이블 상태:
레코드 수: 600
파일 수: 15
스냅샷 수: 13

파일 구조:
├── data/
│   ├── order_date_month=2024-01/
│   │   ├── 00000-100-71e3fd79-15d4-4f60-bd25-c332d0c61d71-00001.parquet  (1.8 KB)
│   │   ├── 00000-113-e944a71e-5121-4e8d-afb8-37abb4a9167b-00001.parquet  (3.6 KB)
│   │   ├── 00000-120-5afec7b2-bc83-41bf-a590-06dd94582164-00001.parquet  (3.6 KB)
│   │   ├── 00000-131-075482d4-9812-4903-9fb1-0375f5663e74-00001.parquet  (3.9 KB)
│   │   ├── 00000-69-6b680a7c-560e-4038-8c12-bdb7ae56aed9-00001.parquet  (3.0 KB)
│   │   └── 00000-75-4557e7d1-e2e2-4063-b7a9-bdc43bc7a0d7-00001.parquet  (2.8 KB)
│   ├── order_date_month=2024-02/
│   │   ├── 00000-113-e944a71e-5121-4e8d-afb8-37abb4a9167b-00002.parquet  (3.9 KB)
│   │   ├── 00000-131-075482d4-9812-4903-9fb1-0375f5663e74-00002.parquet  (4.2 KB)
│   │   ├── 00000-81-28b87232-ee0b-49b7-9eb7-32eddef66fe2-00001.parquet  (3.1 KB)
│   │   └── 00000-87-d0f8bf21-cded-49f6-8166-8d245926ea36-00001.parquet  (2.9 KB)
│   └── order_date_month=2024-

---
## Phase 6: 장애 복구 (Time Travel)

### 시나리오: 실수로 데이터 대량 삭제

운영자가 실수로 잘못된 DELETE 쿼리를 실행하여 데이터가 삭제되었습니다. Time Travel로 복구합니다.

In [62]:
# 삭제 전 레코드 수 기록
count_before_accident = spark.sql(f"SELECT COUNT(*) FROM {TABLE_NAME}").collect()[0][0]
print(f"삭제 전 레코드 수: {count_before_accident}")

# 현재 스냅샷 ID 기록 (복구 지점)
safe_snapshot = spark.sql(f"""
SELECT snapshot_id FROM {TABLE_NAME}.snapshots
ORDER BY committed_at DESC LIMIT 1
""").collect()[0][0]
print(f"안전한 스냅샷 ID: {safe_snapshot}")

삭제 전 레코드 수: 600
안전한 스냅샷 ID: 5831915893391782548


In [63]:
# 사고 발생: 실수로 Seoul 지역 데이터 전체 삭제!
print("[사고 발생] 실수로 Seoul 지역 데이터 전체 삭제!")
spark.sql(f"DELETE FROM {TABLE_NAME} WHERE region = 'Seoul'")

count_after_accident = spark.sql(f"SELECT COUNT(*) FROM {TABLE_NAME}").collect()[0][0]
deleted = count_before_accident - count_after_accident
print(f"삭제 후 레코드 수: {count_after_accident} ({deleted}건 삭제됨!)")

print("\n현재 region 분포:")
spark.sql(f"SELECT region, COUNT(*) FROM {TABLE_NAME} GROUP BY region").show()

[사고 발생] 실수로 Seoul 지역 데이터 전체 삭제!
삭제 후 레코드 수: 419 (181건 삭제됨!)

현재 region 분포:
+------+--------+
|region|count(1)|
+------+--------+
|  Jeju|     228|
| Busan|     191|
+------+--------+



In [64]:
# 복구 방법 1: Time Travel로 삭제 전 데이터 확인
print(f"[복구] 스냅샷 {safe_snapshot}의 데이터 확인:")
safe_count = spark.sql(f"""
SELECT COUNT(*) FROM {TABLE_NAME}
VERSION AS OF {safe_snapshot}
""").collect()[0][0]
print(f"스냅샷 {safe_snapshot} 레코드 수: {safe_count}")

print("\n스냅샷의 region 분포:")
spark.sql(f"""
SELECT region, COUNT(*) as count
FROM {TABLE_NAME} VERSION AS OF {safe_snapshot}
GROUP BY region
""").show()

[복구] 스냅샷 5831915893391782548의 데이터 확인:
스냅샷 5831915893391782548 레코드 수: 600

스냅샷의 region 분포:
+------+-----+
|region|count|
+------+-----+
| Seoul|  181|
|  Jeju|  228|
| Busan|  191|
+------+-----+



In [65]:
# 복구 방법 2: Rollback으로 테이블 상태를 이전 스냅샷으로 되돌리기
print("[복구 실행] Rollback to safe snapshot...")
spark.sql(f"""
CALL demo.system.rollback_to_snapshot(
    table => '{TABLE_NAME}',
    snapshot_id => {safe_snapshot}
)
""")

count_after_recovery = spark.sql(f"SELECT COUNT(*) FROM {TABLE_NAME}").collect()[0][0]
print(f"\n복구 완료!")
print(
    f"레코드 수: {count_before_accident} → {count_after_accident} (사고) → {count_after_recovery} (복구)"
)

print("\n복구 후 region 분포:")
spark.sql(f"SELECT region, COUNT(*) FROM {TABLE_NAME} GROUP BY region").show()

[복구 실행] Rollback to safe snapshot...

복구 완료!
레코드 수: 600 → 419 (사고) → 600 (복구)

복구 후 region 분포:
+------+--------+
|region|count(1)|
+------+--------+
| Seoul|     181|
|  Jeju|     228|
| Busan|     191|
+------+--------+



### 관찰 포인트 — 장애 복구

- `VERSION AS OF`로 삭제 전 데이터를 **먼저 확인**한 후 복구할 수 있었습니다
- `rollback_to_snapshot`으로 테이블 상태를 **원래대로 되돌렸습니다**
- Rollback은 데이터를 복사하지 않고 **메타데이터 포인터만 변경**하므로 즉시 완료됩니다
- 단, `expire_snapshots`로 삭제된 스냅샷으로는 복구할 수 없으므로 **보존 기간 설정이 중요**합니다

---
## 전체 파이프라인 요약

```
Phase 1: 일별 배치 적재        → INSERT (append)
Phase 2: 상태 업데이트          → UPDATE (COW)
Phase 3: 스키마 변경           → ALTER TABLE ADD COLUMN
Phase 4: 컴팩션               → rewrite_data_files (sort)
Phase 5: 스냅샷 정리           → expire_snapshots + rewrite_manifests
Phase 6: 장애 복구             → VERSION AS OF + rollback_to_snapshot
```

### 운영 체크리스트

| 항목 | 주기 | 명령 |
|------|------|------|
| 데이터 적재 | 매일 | `df.writeTo(table).append()` |
| 컴팩션 | 매일 | `rewrite_data_files(strategy => 'sort')` |
| 스냅샷 만료 | 매일~주간 | `expire_snapshots(retain_last => N)` |
| 고아 파일 정리 | 주간~월간 | `remove_orphan_files(dry_run => true)` |
| Manifest 재작성 | 주간 | `rewrite_manifests()` |
| 파티션 모니터링 | 주간 | `SELECT * FROM table.partitions` |
| 파일 크기 모니터링 | 주간 | `SELECT * FROM table.files` |

In [66]:
# 최종 테이블 상태
print("=" * 60)
print("최종 테이블 상태")
print("=" * 60)
print(f"레코드 수: {spark.sql(f'SELECT COUNT(*) FROM {TABLE_NAME}').collect()[0][0]}")
print(f"파일 수: {count_files(TABLE_PATH)}")
print(f"총 크기: {total_size(TABLE_PATH):,} bytes")
print(f"스냅샷 수: {len(spark.sql(f'SELECT * FROM {TABLE_NAME}.snapshots').collect())}")

print("\n스키마:")
spark.sql(f"DESCRIBE {TABLE_NAME}").show()

print("파티션:")
spark.sql(f"SELECT * FROM {TABLE_NAME}.partitions").show(truncate=False)

최종 테이블 상태
레코드 수: 600
파일 수: 18
총 크기: 470,164 bytes
스냅샷 수: 14

스키마:
+--------------+------------------+-------+
|      col_name|         data_type|comment|
+--------------+------------------+-------+
|      order_id|            bigint|   null|
|   customer_id|            bigint|   null|
|  product_name|            string|   null|
|    order_date|              date|   null|
|        amount|     decimal(10,2)|   null|
|        status|            string|   null|
|        region|            string|   null|
|              |                  |       |
|# Partitioning|                  |       |
|        Part 0|months(order_date)|       |
+--------------+------------------+-------+

파티션:
+---------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+-----------------------+------------------------+
|partition|spec_id|record_count|file_count|total_data_file_size_in_bytes|posi

In [67]:
spark.stop()
print("Spark 세션 종료")
print("\nEnd-to-End 시나리오 실습 완료!")

Spark 세션 종료

End-to-End 시나리오 실습 완료!
